In [1]:
import numpy as np
from collections import defaultdict as dd
from typing import List, Dict, Set
import nltk
from tqdm import tqdm_notebook, tnrange
import pickle
from termcolor import colored

In [2]:
def load_word_set():
    all_lemmas = set()
    for line in open('Dane/polimorfologik-2.1.txt', encoding='utf-8'):
        L = line.split(';')[1].lower()
        all_lemmas.add(L)
    return all_lemmas

In [3]:
def get_normal_form(w):
    polish_map = {
        "ż": "z",
        "ź": "z",
        "x": "z",
        "ó": "o",
        "ł": "l",
        "ć": "c",
        "ą": "a",
        "ń": "n",
        "ę": "e",
    }
    ortographic_map = {
        "u": "o",
        "om": "a",
        "en": "e",
        "em": "e",
        "on": "a",
        "rz": "e",
#         "fk": "wk",
#         "af": "aw",
        "f": "w",
        "sz": "z",
    }
    tmp = ""
    for c in w:
        tmp += c if c not in polish_map else polish_map[c]
    l = len(w)
    res = ""
    i = 0
    while i < l:
        if i < l - 1 and tmp[i:(i+2)] in ortographic_map:
            res += ortographic_map[tmp[i:(i+2)]]
            i += 2
            continue
        if tmp[i] in ortographic_map:
            res += ortographic_map[tmp[i]]
            i += 1
            continue
        res += tmp[i]
        i += 1
    return res

In [4]:
def get_dictionaries():
    ws = load_word_set()
    res1 = dd(list)
    res2 = dd(list)
    for w in word_set:
        norm_w = get_normal_form(w)
        res1[norm_w].append(w)
        res2[w[0]].append(w)
    return res1, res2

In [5]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])

In [6]:
def argmin(l):
    res = None
    for i, el in enumerate(l):
        if res is None or el < l[res]:
            res = i
    return res
            

In [12]:
def find_closest(w, words):
    edit_distances = [levenshtein(w, w1) for w1 in words]
    return words[argmin(edit_distances)]

In [13]:
def correct_word(w, norm_dicts):
    d1, d2 = norm_dicts
    norm_w = get_normal_form(w)
    if norm_w in d1:
        return find_closest(w, d1[norm_w])
    w_candidates = d2[w[0]][:]
    if len(w) > 1:
        w_candidates += d2[w[1]]
    return find_closest(w, w_candidates)

In [14]:
word_set = load_word_set()

In [ ]:
d1, d2 = get_dictionaries()
with open("Dane/literowki1.txt", "r") as ifile:
    for line in ifile:
        line = nltk.word_tokenize(line)
        correct, incorrect = line[0], line[1]
        print(f"{line[0]}: {correct_word(line[1], (d1, d2))}")

lokomotywa: lokomotywa
lokomotywa: kolorowa
lokomotywa: lokomotywa
lokomotywa: kolorowa
lokomotywa: lokowała
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdopodobieństwo: prawdopodobieństwo
prawdomówny: prawdomówno
prawdomówny: prawdomówny
prawdomówny: przodowny
komputerek: kompaturek
komputerek: komputerem
komputerek: komputerek
komputerek: korektorek
komputerek: kompaturek
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyna
dziewczyna: dziewczyn
dziewczyna: dziewczyn
przyjaźń: przyjada
przyjaźń: przyjaźń
przyjaźń: przyjaźń
przyjaźń: przyjaźń
przyjaźń: przyjaźń
niszczyciel: 